In [34]:
import numpy as np
import pandas as pd
import json
import matplotlib

In [35]:
with open('/home/kimino/soft/qcm-detector/results.json', 'r') as results_file:
    r_json = json.loads(results_file.read())

In [36]:
experiments = [r_json[exp] for exp in r_json]

In [68]:
def get_tf_positives(experiment, experiment_name):
    true_positives = experiment['True Positives']
    false_positives = [exp for exp in experiment['Detections'] if exp not in experiment['True Positives']]
    tp_df = pd.DataFrame(true_positives)
    fp_df = pd.DataFrame(false_positives)
    tp_df['is_correct'] = True
    fp_df['is_correct'] = False
    df = pd.concat([tp_df, fp_df], ignore_index= True)
    df['experiment'] = experiment_name
    df['path'] = experiment['Path']
    return df
def get_all_detections(experiment_json):
    detection_df = pd.DataFrame()
    for experiment in experiment_json:
        tf_df = get_tf_positives(experiment_json[experiment], experiment)
        detection_df = pd.concat([detection_df, tf_df], ignore_index= True, sort = False)
    return detection_df

In [69]:
results_df = get_all_detections(r_json)

ipykernel_launcher:8: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.



In [70]:
results_df.groupby(['detector', 'is_correct']).count()

timestamp  frequency  resistance  period  time  \
detector        is_correct                                                   
IsolationForest False              46         46          46      46    46   
                True               46         46          46      46    46   
Mean-shift      False               2          2           2       2     2   
                True             1222       1222        1222    1222  1222   
OneClassSVM     False             531        531         531     531   531   
                True              288        288         288     288   288   

                            severity  experiment  path  
detector        is_correct                              
IsolationForest False             46          46    46  
                True              46          46    46  
Mean-shift      False              2           2     2  
                True            1222        1222  1222  
OneClassSVM     False            531         531   531  
                True             288         288   288

In [71]:
mean_shift_detection = results_df[results_df['detector'] == 'Mean-shift']

In [72]:
mean_shift_detection[results_df['is_correct'] == False]

ipykernel_launcher:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.


,timestamp,frequency,resistance,period,time,severity,detector,is_correct,experiment,path
769,1.608135e+09,5009754.4,412.63,"[0, 0]",,mild,Mean-shift,False,Insertion-30ppm,/home/kimino/soft/qcm-detector/data/tests/Inse...
2059,1.608139e+09,5009701.0,424.00,"[0, 0]",,mild,Mean-shift,False,Insertion-100ppm,/home/kimino/soft/qcm-detector/data/tests/Inse...


In [73]:
mean_shift_detection['Path']

KeyError: 'Path'

0        True
1        True
2        True
3        True
4        True
        ...  
2130    False
2131    False
2132    False
2133    False
2134    False
Name: detector, Length: 2135, dtype: bool